# dynamo query table

In [1]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr
# from tweet_grab

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

client = boto3.client('dynamodb',
  # endpoint_url = "http://xanaxprincess.asuscomm.com:8001",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            # endpoint_url = "http://xanaxprincess.asuscomm.com:8001", \
            region_name='us-east-2')

tweets = dynamodb.Table("tweets")
print(tweets)
    
print("imported modules.")

dynamodb.Table(name='tweets')
imported modules.


## query

In [2]:
query = """
select * from tweets
"""

print("query: {}".format(query))

query: 
select * from tweets



In [3]:
os.getcwd()

'/home/steven/Documents/twitter.ai/notebooks'

## tweet_grab library

In [4]:
os.chdir('/home/steven/Documents/twitter.ai/src/main/')

In [5]:
from tweet_grab import *
import tweet_grab
print("imported tweet_grab.")

imported modules.
read creds.json
imported tweet_grab.


In [6]:
print(api)

NameError: name 'api' is not defined

## get following

In [7]:
try:
    my_following = tweet_grab.get_followers("xanax_princess_")

except Exception as e:
    print("Exception: {}".format(e))

read creds.json
got most recent 100 following
got next 100 following


In [8]:
# tweets.query(KeyConditionExpression=Key('user'))

In [9]:
user = "xanax_princess_"
df = tweets.query(KeyConditionExpression=Key('user').eq(user))
df["Count"]

0

In [10]:
response = tweets.scan()
response["Count"]

5552